# Exercises

This is the notebook containing the exercises for Feature Store, Model Monitor, and Clarify. Tested for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [1]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this exercise we'll work with a wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality/

```P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.```

In [2]:
import pandas as pd
from sklearn import datasets
import time
import uuid

In [3]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']

If we leave the column names as-is, Feature Store won't be able to handle the `/` in `od280/od315_of_diluted_wines` (`/` is a delimiter Feature Store uses to manage how features are organized.)

In [4]:
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [5]:
# Add a feature group
df["EventTime"] = time.time()
df["id"] = range(len(df))

# TODO: Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name="winedata-features", sagemaker_session=session
)

# TODO: Load Feature definitions
feature_group.load_feature_definitions(data_frame= df)

[FeatureDefinition(feature_name='alcohol', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='malic_acid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='alcalinity_of_ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='magnesium', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='total_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='flavanoids', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='nonflavanoid_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collec

The feature group is not created until we call the `create` method, let's do that now:

In [6]:
# Create the feature store:


In [7]:
# TODO
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name='id',
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # TODO                                                                                       │
│ ❱ 2 feature_group.create(                                                                        │
│   3 │   s3_uri=f"s3://{bucket}/features",                                                        │
│   4 │   record_identifier_name='id',                                                             │
│   5 │   event_time_feature_name="EventTime",                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/feature_store/feature_group.py:789 in create   │
│                                                                                                  │
│    786 │   │   │   │   {"offline_store_config": offline_store_config.to_dict()}                  │
│    787 │   │   │   )                                                                             │
│    788 │   │                                                                                     │
│ ❱  789 │   │   return self.sagemaker_session.create_feature_group(**create_feature_store_args)   │
│    790 │                                                                                         │
│    791 │   def delete(self):                                                                     │
│    792 │   │   """Delete a FeatureGroup."""                                                      │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:6252 in create_feature_group        │
│                                                                                                  │
│   6249 │   │   │   Description=description,                                                      │
│   6250 │   │   │   Tags=tags,                                                                    │
│   6251 │   │   )                                                                                 │
│ ❱ 6252 │   │   return self.sagemaker_client.create_feature_group(**kwargs)                       │
│   6253 │                                                                                         │
│   6254 │   def describe_feature_group(                                                           │
│   6255 │   │   self,                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/client.py:

Lastly, ingest some data into your feature group:

In [ ]:
# TODO
feature_group.ingest(data_frame=df, max_workers=3, wait=True)

In [8]:
runtime = session.boto_session.client(
  'sagemaker-featurestore-runtime',
  region_name=region
)

data = runtime.get_record(
    FeatureGroupName="winedata-features",
    RecordIdentifierValueAsString="0"
)

In [9]:
data

{'ResponseMetadata': {'RequestId': 'ffc94a4a-46f2-48de-9222-c558936d915c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ffc94a4a-46f2-48de-9222-c558936d915c',
   'content-type': 'application/json',
   'content-length': '1216',
   'date': 'Tue, 11 Mar 2025 06:41:26 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'alcohol', 'ValueAsString': '14.23'},
  {'FeatureName': 'malic_acid', 'ValueAsString': '1.71'},
  {'FeatureName': 'ash', 'ValueAsString': '2.43'},
  {'FeatureName': 'alcalinity_of_ash', 'ValueAsString': '15.6'},
  {'FeatureName': 'magnesium', 'ValueAsString': '127.0'},
  {'FeatureName': 'total_phenols', 'ValueAsString': '2.8'},
  {'FeatureName': 'flavanoids', 'ValueAsString': '3.06'},
  {'FeatureName': 'nonflavanoid_phenols', 'ValueAsString': '0.28'},
  {'FeatureName': 'proanthocyanins', 'ValueAsString': '2.29'},
  {'FeatureName': 'color_intensity', 'ValueAsString': '5.64'},
  {'FeatureName': 'hue', 'ValueAsString': '1.04'},
  {'FeatureName': 'od280_

Great job! You've demonstrated your understanding of creating feature groups and ingesting data into them using Feature Store. Next up we'll cover Model Monitor!

## Model Monitor

In this exercise we'll create a monitoring schedule for a deployed model. We're going to provide code to help you deploy a model and get started, so that you can focus on Model Monitor for this exercise. __Remember to clean up your model before you end a work session__. We'll provide some code at the end to help you clean up your model. We'll begin by reloading our data from the previous exercise.



In [10]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [11]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [12]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [13]:
algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/wine_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

[03/11/25 06:41:37] INFO     Ignoring unnecessary instance type: None.                            ]8;id=497428;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=937595;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=24626;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=684465;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: xgboost-2025-03-11-06-41-37-858       ]8;id=191821;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=714341;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-11 06:41:38 Starting - Starting the training job......
..25-03-11 06:42:36 Downloading - Downloading input data.
.....03-11 06:43:07 Downloading - Downloading the training image.
Arguments: training - Training image download completed. Training in progress..
[2025-03-11:06:44:11:INFO] Running standalone xgboost training.
[2025-03-11:06:44:11:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8551.41mb
[2025-03-11:06:44:11:INFO] Determined delimiter of CSV input is ','
[06:44:11] S3DistributionType set as FullyReplicated
[06:44:11] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-03-11:06:44:11:INFO] Determined delimiter of CSV input is ','
[06:44:11] S3DistributionType set as FullyReplicated
[06:44:11] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[06:44:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roo

Now that your training job has finished, you can perform the first task in this exercise: creating a data capture config. Configure your model to sample `34%` of inferences:

In [14]:
# TODO
from sagemaker.model_monitor import DataCaptureConfig

capture_uri = f's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=capture_uri
)

Great! We'll use your config to deploy a model below:

In [33]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

[03/11/25 07:14:50] INFO     Creating model with name: xgboost-2025-03-11-07-14-50-634              ]8;id=710355;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=779186;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/11/25 07:14:51] INFO     Creating endpoint-config with name xgboost-2025-03-11-07-14-50-634     ]8;id=603998;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=234071;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name xgboost-2025-03-11-07-14-50-634            ]8;id=712527;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=421418;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\

-------!

Great! You should see an indicator like this when the deployment finishes:

```
-----------------!
```
We can test your deployment like so:

In [16]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
inputs = test.copy()
# Drop the target variable
inputs = inputs.drop(columns=inputs.columns[0])
x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')
x_pred

'0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388'

All systems go! To finish up the exercise, we're going to provide you with a DefaultModelMonitor and a suggested baseline. Combine the `xgb_predictor` and the provided `my_monitor` to configure the monitoring schedule for _hourly_ monitoring.

In [31]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

[03/11/25 07:10:13] INFO     Ignoring unnecessary instance type: None.                            ]8;id=36053;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=704107;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [32]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)

[03/11/25 07:10:17] INFO     Creating processing-job with name                                      ]8;id=375872;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=358189;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             baseline-suggestion-job-2025-03-11-07-10-17-624                                       

............2025-03-11 07:12:14.590075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-11 07:12:14.590103: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-03-11 07:12:16.169769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-11 07:12:16.169800: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-11 07:12:16.169824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-216-163.ec2.internal): /proc/driver/nvidia/version does not exist
2025-03-11 07:12:16.170085: I tensor

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 my_monitor.suggest_baseline(                                                                 │
│   2 │   baseline_dataset=f's3://{bucket}/data/train.csv',                                        │
│   3 │   dataset_format=DatasetFormat.csv(header=False),                                          │
│   4 )                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py:1922 in      │
│ suggest_baseline                                                                                 │
│                                                                                                  │
│   1919 │   │   │   if baseline_job_input is not None                                             │
│   1920 │   │   ]                                                                                 │
│   1921 │   │                                                                                     │
│ ❱ 1922 │   │   baselining_processor.run(                                                         │
│   1923 │   │   │   inputs=baseline_job_inputs,                                                   │
│   1924 │   │   │   outputs=[normalized_baseline_output],                                         │
│   1925 │   │   │   arguments=self.arguments,                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/processing.py:278 in run                       │
│                                                                                                  │
│    275 │   │   )                                                                                 │
│    276 │   │   self.jobs.append(self.latest_job)                                                 │
│    277 │   │   if wait:                                                                          │
│ ❱  278 │   │   │   self.latest_job.wait(logs=logs)                                               │
│    279 │                                                                                         │
│    280 │   def _extend_processing_args(self, inputs, outputs, **kwargs):  # pylint: disable=W06  │
│    281 │   │   """Extend inputs and outputs based on extra parameters"""                         │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/processin

Below, provide the monitoring schedule:

In [ ]:
# TODO
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='my-monitoring-schedule2',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

Great job! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you configured Model Monitor to watch a simple model. Next, we'll monitor the same deployment for explainability.

__REMINDER:__ Don't leave your model deployed overnight. If you aren't going to follow up with the Clarify exercise within a few hours, use the code below to remove your model:

In [21]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()

[03/11/25 07:01:30] INFO     Ignoring unnecessary instance type: None.                            ]8;id=796791;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=741625;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Deleting Monitoring Schedule with name: my-monitoring-schedule1        ]8;id=814320;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=437321;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#2237\2237]8;;\

[03/11/25 07:01:46] INFO     Deleting Data Quality Job Definition with name:               ]8;id=680574;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=244917;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py#2461\2461]8;;\
                             data-quality-job-definition-2025-03-11-07-01-13-032                                   

In [22]:
xgb_predictor.delete_endpoint()

                    INFO     Deleting endpoint configuration with name:                             ]8;id=571700;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=886207;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4865\4865]8;;\
                             xgboost-2025-03-11-06-45-32-377                                                       

                    INFO     Deleting endpoint with name: xgboost-2025-03-11-06-45-32-377           ]8;id=33779;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=648356;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4855\4855]8;;\

## Clarify

For the last exercise we'll deploy an explainability monitor using Clarify. We're going to use the model that you deployed in the last exercise, but if you cleaned up your deployments from the previous exercise, that's ok! You can rerun the deployment from the previous exercise up to the point where we deployed our model. It'll look like this:

```python
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
```

Once your model is deployed, you can come back here. _REMINDER_: you need to clean up your deployment, don't leave it running overnight. We'll provide some code at the end to delete your deployment.

## Prep

We'll begin by reloading our data from the previous exercise.

In [23]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [24]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [25]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

Great! Our data is staged and our model is deployed - let's monitor it for explainability. We need to define three config objects, the `SHAPConfig`, the `ModelConfig`, and the `ExplainabilityAnalysisConfig`. Below, we provide the `SHAPConfig`.

In [26]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)

Next up, fill in the blanks to define the `ModelConfig` and `ExplainabilityAnalysisConfig`.

Before we apply our config, we need to create the monitor object. This is what we'll apply all our config to.

In [27]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)

[03/11/25 07:04:54] INFO     Ignoring unnecessary instance type: None.                            ]8;id=970541;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=179666;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [34]:
# TODO
model_config = sagemaker.clarify.ModelConfig(
    model_name="xgboost-2025-03-11-07-14-50-634",
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="text/csv",
    accept_type="text/csv",
)
# ...

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
    explainability_config=shap_config,
        model_config=model_config,
        headers=train.columns.to_list()[1:],
    )
# ...

Everything's ready! Below, create a monitoring schedule using the configs we created. Set the schedule to run _daily_.

In [35]:
# TODO 
from sagemaker.model_monitor import CronExpressionGenerator


explainability_uri = f"s3://{bucket}/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
# ...
)

[03/11/25 07:18:52] INFO     Uploading analysis config to {s3_uri}.                 ]8;id=881576;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/clarify_model_monitoring.py\clarify_model_monitoring.py]8;;\:]8;id=130087;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/clarify_model_monitoring.py#227\227]8;;\

[03/11/25 07:18:53] INFO     Creating Monitoring Schedule with name:                       ]8;id=635118;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=719693;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py#1560\1560]8;;\
                             monitoring-schedule-2025-03-11-07-18-52-840                                           

Way to go! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you deployed a monitor for explainability to your SageMaker endpoint. This is the last exercise - you'll apply these learnings again in your Project at the end of the course.



__REMINDER:__ Don't leave your model deployed overnight. Use the code below to remove your model:

In [36]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()

[03/11/25 07:18:54] INFO     Ignoring unnecessary instance type: None.                            ]8;id=402094;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=732746;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Deleting Monitoring Schedule with name:                                ]8;id=56553;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=882231;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#2237\2237]8;;\
                             monitoring-schedule-2025-03-11-07-18-52-840                                           

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 monitors = xgb_predictor.list_monitors()                                                     │
│   2 for monitor in monitors:                                                                     │
│ ❱ 3 │   monitor.delete_monitoring_schedule()                                                     │
│   4                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/clarify_model_monitoring.py:1284 │
│ in delete_monitoring_schedule                                                                    │
│                                                                                                  │
│   1281 │                                                                                         │
│   1282 │   def delete_monitoring_schedule(self):                                                 │
│   1283 │   │   """Deletes the monitoring schedule and its job definition."""                     │
│ ❱ 1284 │   │   super(ModelExplainabilityMonitor, self).delete_monitoring_schedule()              │
│   1285 │   │   # Delete job definition.                                                          │
│   1286 │   │   message = "Deleting Model Explainability Job Definition with name: {}".format(    │
│   1287 │   │   │   self.job_definition_name                                                      │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py:643 in       │
│ delete_monitoring_schedule                                                                       │
│                                                                                                  │
│    640 │   def delete_monitoring_schedule(self):                                                 │
│    641 │   │   """Deletes the monitoring schedule (subclass is responsible for deleting job def  │
│    642 │   │   # DO NOT call super which erases schedule name and makes wait impossible.         │
│ ❱  643 │   │   self.sagemaker_session.delete_monitoring_schedule(                                │
│    644 │   │   │   monitoring_schedule_name=self.monitoring_schedule_name                        │
│    645 │   │   )                                                                                 │
│    646 │   │   if self.job_definition_name is not None:                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:2238 in delete_monitoring_schedule  │
│                                                                                                  │
│   2235 │   │   │   │   Schedule to delete.                                                       │
│   2236 │   │   """                                                                               │
│   2237 │   │   logger.info("Deleting Monitoring Schedule with name: %s", monitoring_schedule_na  │
│ ❱ 2238 │   │   self.sagemaker_client.delete_monitoring_schedule(                                 │
│   2239 │   │   │   MonitoringScheduleName=monitoring_schedule_name                               │
│   2240 │   │   )                                                                                 │
│   2241                                                                                           │
│                                                            

In [127]:
xgb_predictor.delete_endpoint()